In [1]:
import sys
sys.path.append('./src')

from utils import *
from osqp_solver import *
from osqp_sqp import *
from osqp_mpc import *
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import meshcat.geometry as g
import meshcat.transformations as tf
import numpy as np
np.set_printoptions(linewidth=99999999)
    
model_dir = "description/"
urdf_path = model_dir + "indy7.urdf"
model, visual_model, collision_model = load_robot_model(urdf_path, model_dir)

# Setup pinocchio visualizer
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer()
viz.loadViewerModel()

# Create MPC solver and optimizer
solver = OSQPSolver(model)
sqp = SQP_OSQP(solver)
controller = MPC_OSQP(model, sqp, solver)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/


In [2]:
# initial state - zeros
xstart = np.ones(solver.nx)

# goal ee positions
endpoints = np.array([
    #np.array(controller.eepos(np.zeros(controller.nq))),
    np.array(solver.eepos(0.3*np.ones(6))),
    np.array(solver.eepos(0.9*np.ones(6))),
    #np.array(controller.eepos(np.zeros(controller.nq))), 
    np.array(solver.eepos(np.array([-3.0, 1.0, 1.0, 1.0, 0.0, 0.0])))
])

print("Endpoints:")
print(endpoints)

# Visualize goals
for i, p in enumerate(endpoints):
    viz.viewer[f'point{i}'].set_object(
        g.Sphere(0.05), 
        g.MeshLambertMaterial(color=0xff0000))

    T = tf.translation_matrix(np.array(p[:]))
    viz.viewer[f'point{i}'].set_transform(T)

Endpoints:
[[-0.34013996 -0.30723899  1.15448739]
 [-0.33030096 -0.77069748  0.53591299]
 [ 0.89016846  0.23030064  0.46709395]]


In [3]:
# Run MPC
q_trajectory = controller.run_mpc(xstart, endpoints)

0.8822365440884666
0.8932961725209858
0.9053590870084657
0.9169408371914585
0.9255210446353558
0.9282393131207549
0.9225972253243112
0.906656225026087
0.8807399644231377
0.8476509587262862
0.8094913693504235
0.76770957060316
0.7228396855988445
0.6758283420304312
0.6277886425897673
0.5796551655338105
0.5323310640882435
0.48671071260627696
0.44375969396250486
0.40409175592632124
0.3681653755561697
0.3366494243798342
0.3101692591565575
0.2894280149515967
0.2747799069516257
0.2662859174807664
0.2636137187208512
0.26612094880423526
0.2728789201801662
0.28277992034133564
0.29472704639951747
0.30792313664714976
0.32140517862502965
0.3342395813945217
0.3464466026896213
0.3582413714902138
0.3695792089094418
0.380338450300856
0.39052988776085457
0.40007179053294467
0.4088790024871386
0.41689801347930205
0.42409847563440617
0.4303555855050925
0.43570457068347584
0.44012631066655344
0.4438069059989586
0.4466944718557435
0.44875179178181296
0.45011180642483617
0.4509663634861323
0.45117189391222146

In [ ]:
print_stats(sqp.get_stats())
viz.play(q_trajectory=q_trajectory, dt=0.01)

In [ ]:
# Setup pinocchio visualizer
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer()
viz.loadViewerModel()
# Visualize goals
for i, p in enumerate(endpoints):
    viz.viewer[f'point{i}'].set_object(
        g.Sphere(0.05), 
        g.MeshLambertMaterial(color=0xff0000))

    T = tf.translation_matrix(np.array(p[:]))
    viz.viewer[f'point{i}'].set_transform(T)
#print_stats(controller.solver.get_stats())
while 1:
    viz.play(q_trajectory=q_trajectory, dt=0.05)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7011/static/


KeyboardInterrupt: 